In [ ]:
import h5py
import os

from tqdm import tqdm
from natsort import natsorted
from pathlib import Path
from EvEye.utils.processor.TxtProcessor import TxtProcessor
from EvEye.utils.processor.HDF5Processor import HDF5Processor
from EvEye.utils.tonic.functional.ToFrameStack import to_frame_stack_numpy
from EvEye.utils.tonic.slicers.SliceEventsAtIndices import slice_events_at_timepoints

In [ ]:
events_base_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeCenterDataset/val/data"
)
ellipses_base_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeEllipseLabels"
)
output_path = Path("/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeEllipseDataset")
events_path = natsorted(list(events_base_path.glob("*.txt")))
ellipses_path = natsorted(list(ellipses_base_path.glob("*.txt")))

In [ ]:
time_window = 10000
sensor_size = (346, 260, 2)
events_interpolation = "causal_linear"

In [ ]:
for event_path in tqdm(events_path):
    file_name = event_path.stem[:-7]
    ellipse_path = f"{ellipses_base_path}/{file_name}.txt"
    events = TxtProcessor(event_path).load_events_from_txt()
    ellipses = TxtProcessor(ellipse_path).load_ellipses_from_txt()
    event_frame_list = []
    for ellipse in ellipses:
        end_event_time = ellipse['t']
        start_event_time = end_event_time - time_window
        event_segment = slice_events_at_timepoints(
            events, start_event_time, end_event_time
        )
        event_frame = to_frame_stack_numpy(
            event_segment,
            sensor_size,
            1,
            events_interpolation,
            start_event_time,
            end_event_time,
        )
        event_frame_list.append(event_frame)

    with h5py.File(f"{output_path}/{file_name}.h5", "w") as file:
        file.create_dataset("events", data=event_frame_list)
        file.create_dataset("ellipses", data=ellipses)

In [ ]:
h5_path = "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeEllipseDataset/user38_right_session_1_0_1.h5"
events = HDF5Processor(h5_path).read_data("events")[0]

In [ ]:
events.shape